# SIR Analysis


In [1]:
import numpy as np
import pandas as pd
from scipy.integrate import ode, solve_ivp
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

P-V-H models where
- P is the number of pathogens
- V is the number of vectors (ticks)
- H is the number of Hosts (Humans, Deer, Rabbits)

## Host population Equations

$$\frac{dN}{dt} = \beta \left(\frac{K-N}{K}\right) N - bN$$

Where 
 - $N$ is the host population 
 - $K$ is the carrying capacity of the environment
 - $\beta$ is the growth rate of the host

|         | Human | Deer | Rabbit |
| ------- | ----- | ---- | ------ |
| N       | |||
| K       | |||
| $\beta$ | |||

Notes:

I don't think it makes sense for us to use the total human population in the US for this analysis b/c a lot of people in the cities never encounter a tick. As a result I would argue that we should use National Park data and hunting data to get an idea of relative trends of how many people are out in environments where they are exposed to ticks.

## Vector Population Equations

$$ \frac{dV}{dt} =  \hat{\beta} \left(\frac{MN - V}{MN}\right) V - \hat{b}V$$

Where
 - $V$ is the Vector population
 - $\hat{\beta}$ is the density dependent vector birth/death rate
 - $M$ is the number of ticks per host
 - $N$ is the number of hosts
 - $b$ is the density independent death rate

 |         | Human | Deer | Rabbit |
| ------- | ----- | ---- | ------ |
| N       | |||
| M       | |||

## Transmission
 - 3 types
    - Vector Borne: 
        - infected host -> susceptible tick
        - infected tick -> susceptible host
    - Transovarial
        - Mom tick -> children ticks
    - Transstadial
        - ticks stay infected as they go through their life cycles
    - Cofeeeding
        - infected tick -> susceptible tick (while feeding on an uninfected host) 

Uninfected Hosts: $N_s$
Infected Hosts: $Y_{pathogens}$

Uninfected Vectors (ticks): $V_s$
Infected Vectors: $X_{pathogens}$

![Infection Flow Diagram](./infectionFlowDiagram2.jpg)

### Tick to Host Transmission Modelling

#### Infection of Host via Tick Component

$$A_{i} \left(\frac{N - Y_1 - Y_2 - Y_{12}}{N}\right) (X_{i} + X_{12})$$

Where:
 - $A_{i}$ is the rate of transmission from tick to host
    - includes biting rate, probability of transmission, and proportion of hosts to ticks
 - $\left(\frac{N - Y_1 - Y_2 - Y_{12}}{N}\right)$ is the uninfected population divided by total population
 - $(X_{i} + X_{12})$ is the number of vectors infected with pathogen i (including the coinfected vectors)
 

#### Overall Host Infection Equation (Single Pathogen)

$$\frac{dY_1}{dt} = A_1 \left(\frac{N - Y_1 - Y_2 - Y_{12}}{N}\right) (X_1 + X_12)
 + v_{12,1} Y_{12}
 - A_{12} \left(\frac{Y_1}{N}\right) \left(X_2 + X_12\right)
 - \beta \left(\frac{N Y_1}{K}\right)
 - (b + v_1) Y_1
 $$



## Extending the 2-1-1 model to 2-2-1

In order to extend the 2-1-1 model to a 2-2-1 model we need to add in another tick population. To represent this additional tick population we are going to make the following notation change.

In the 2-1-1 model the total population of ticks was represented by $V$ with a single species of ticks notated with $X$. Where $X_i$ represented the number of ticks infected with pathogen $i$.

In the 2-2-1 model, instead of having a single species of ticks, we have two species of ticks. To take this into account, we are adding two additional terms. $W$ to represent the total number of the second species of ticks, and $Z_i$ to represent the number of $W$ ticks infected with pathogen $i$.

### Overall 2-1-1 Host Infection Equation (Single Pathogen)

$$\frac{dY_1}{dt} = 
 A_1^V \left(\frac{N - Y_1 - Y_2 - Y_{12}}{N}\right) (X_1 + X_12)
 + A_1^W \left(\frac{N - Y_1 - Y_2 - Y_{12}}{N}\right) (Z_1 + Z_12)
 + v_{12 \rightarrow 1} Y_{12}
 - A_{12}^V \left(\frac{Y_1}{N}\right) \left(X_2 + X_12\right)
 - A_{12}^W \left(\frac{Y_1}{N}\right) \left(Z_2 + Z_12\right)
 - \beta \left(\frac{N Y_1}{K}\right)
 - (b + v_1) Y_1
 $$


Where 

$$A_1^V \left(\frac{N - Y_1 - Y_2 - Y_{12}}{N}\right) (X_1 + X_{12})$$

represents the increase in the number of hosts infected with pathogen 1 via ticks of species $V$.
Computed by multiplying a transmission constant $A_1^V$ by the susceptable proportion of the host population $\left(\frac{N - Y_1 - Y_2 - Y_{12}}{N}\right)$ and the number of contagious ticks $(X_1 + X_{12})$

$$+ A_1^W \left(\frac{N - Y_1 - Y_2 - Y_{12}}{N}\right) (Z_1 + Z_{12})$$

represents the increase in the number of hosts solely infected with pathogen 1 via ticks of species $W$.

$$+ v_{12 \rightarrow 1} Y_{12}$$

represents the increase in the number of hosts solely infected with pathogen 1 via the recovery of hosts from the coinfected 1,2 state.

$$- A_{12}^V \left(\frac{Y_1}{N}\right) \left(X_2 + X_12\right)$$

represents the decrease in the number of hosts solely infected with pathogen 1 due to coinfection with pathogen 2 via ticks of species $V$.

$$- A_{12}^W \left(\frac{Y_1}{N}\right) \left(Z_2 + Z_12\right)$$

represents the decrease in the number of hosts solely infected with pathogen 1 due to coinfection with pathogen 2 via ticks of species $W$

$$- \beta \left(\frac{N Y_1}{K}\right)$$

represents the decrease in the number of hosts solely infected with pathogen 1 due to some growth process???

$$- (b + v_1) Y_1$$

represents the decrease in the number of hosts solely infected with pathogen 1 due to host recovery and death.

### Overall 2-2-1 Host Infection Equation (Coinfection)

$$
\frac{dY_{12}}{d_t} = 
A_{12}^V \left(\frac{Y_1}{N}\right) (X_2 + X_{12})
+ A_{12}^W \left(\frac{Y_1}{N}\right) (Z_2 + Z_{12})
+ A_{21}^V \left(\frac{Y_2}{N}\right) (X_1 + X_{12})
+ A_{21}^W \left(\frac{Y_2}{N}\right) (Z_1 + Z_{12})
- \beta \left(\frac{N Y_{12}}{K}\right)
- (b + v_{12 \rightarrow 1} + v_{12 \rightarrow 2}) Y_{12}
$$

Where:

$$A_{12}^V \left(\frac{Y_1}{N}\right) (X_2 + X_{12})$$

is the increase in number of coinfected hosts from hosts that were already infected by pathogen 1 via a tick of species $V$.

$$+ A_{12}^W \left(\frac{Y_1}{N}\right) (Z_2 + Z_{12})$$

is the increase in number of coinfected hosts from hosts that were already infected by pathogen 1 via a tick of species $W$.

$$+ A_{21}^V \left(\frac{Y_2}{N}\right) (X_1 + X_{12})$$

is the increase in number of coinfected hosts from hosts that were already infected by pathogen 2 via a tick of species $V$.

$$+ A_{21}^W \left(\frac{Y_2}{N}\right) (Z_1 + Z_{12})$$

is the increase in number of coinfected hosts from hosts that were already infected by pathogen 2 via a tick of species $W$.

$$- \beta \left(\frac{N Y_{12}}{K}\right)$$

is the decrease in number of coinfected hosts due to some growth dynamic

$$- (b + v_{12 \rightarrow 1} + v_{12 \rightarrow 2}) Y_{12}$$

represents the decrease in the number of coinfected hosts due to death ($b$), recovery to solely being infected with pathogen 1 ($v_{12 \rightarrow 1}), and recovery to solely being infected with pathogen 2 ($v_{12 \rightarrow 2})$)

### Overall 2-2-1 Tick Infection Equation (Single Pathogen)

$$
\frac{dX_1}{d_t} = 
\hat{A}_1^V \left(\frac{Y_1 + Y_{12}}{N}\right) (V - X_1 - X_2 - X_{12})
+ \hat{\beta_t^V} (\gamma_1^V X_1 + \gamma_{12}^V X_{12})
+ \mu_1^V \left(\frac{X_1 + X_{12}}{V}\right) (V - X_1 - X_2 - X_{12})
+ \mu_1^W \left(\frac{Z_1 + Z_{12}}{W}\right) (V - X_1 - X_2 - X_{12})
- \hat{A}_{1 \rightarrow 12}^V \left(\frac{Y_2 + Y_{12}}{N}\right) X_1
- \mu_{1 \rightarrow 12}^V \frac{X_2 + X_{12}}{V} X_1
- \mu_{1 \rightarrow 12}^W \frac{Z2 + Z_{12}}{W} X_1
- \hat{\beta} \frac{V X_1}{M N}
- \hat{b} X_1
$$

Where:

$$\frac{dX_1}{d_t}$$

is the change in the number of ticks of species X that are infected with pathogen 1 only

$$ \hat{A}_1^V \left(\frac{Y_1 + Y_{12}}{N}\right) (V - X_1 - X_2 - X_{12}) $$

is the increase in the number of species X ticks infected solely by pathogen 1 due to interactions with infected hosts

$$+ \hat{\beta_t^V} (\gamma_1^V X_1 + \gamma_{12}^V X_{12})$$

is the increase in the number of species X ticks infected with pathogen 1 only due to trans-ovarial and trans-stadial transmission

$$+ \mu_1^V \left(\frac{X_1 + X_{12}}{V}\right) (V - X_1 - X_2 - X_{12})$$

is the increase in the number of species X ticks infected with pathogen 1 only due to cofeeding with another infected species X tick

$$+ \mu_1^W \left(\frac{Z_1 + Z_{12}}{W}\right) (V - X_1 - X_2 - X_{12})$$

is the increase in the number of species X ticks infected with pathogen 1 only due to cofeeding with anogher infected species W tick

$$- \hat{A}_{1 \rightarrow 12}^V \left(\frac{Y_2 + Y_{12}}{N}\right) X_1$$

is the decrease in the number of species X ticks solely infected with pathogen 1 due to coinfection after feeding on a pathogen 2 infected host

$$- \mu_{1 \rightarrow 12}^V \frac{X_2 + X_{12}}{V} X_1$$

is the decrease in the number of species X ticks solely infected with pathogen 1 due to coinfection via cofeeding with a pathogen 2 infected species V tick.

$$- \mu_{1 \rightarrow 12}^W \frac{Z2 + Z_{12}}{W} X_1$$

is the decrease in the number of species X ticks solely infected with pathogen 1 due to coinfection via cofeeding with a pathogen 2 infected species W tick.

$$- \hat{\beta} \frac{V X_1}{M N}$$

is the decrease in the number of species X ticks solely infected with pathogen 1 due to overall species X population dynamics

$$- \hat{b} X_1$$

is the decrease in the number of species X ticks solely infected with pathogen 1 due to deaths of species X ticks



### Overall 2-2-1 Tick Infection Equation (Coinfection)

$$\frac{dX_{12}}{dt} = 
\hat{A}_{12} \left(\frac{Y_2 + Y_{12}}{N}\right) X_1
+ \hat{A}_{21} \left(\frac{Y_1 + Y_{12}}{N}\right) X_2
+ \mu_{12}^{V} \left(\frac{X_2 + X_{12}}{V}\right) X_1
+ \mu_{21}^{V} \left(\frac{X_1 + X_{12}}{V}\right) X_2
+ \mu_{12}^{W} \left(\frac{Z_2 + Z_{12}}{W}\right) X_1
+ \mu_{21}^{W} \left(\frac{Z_1 + Z_{12}}{W}\right) X_2
- \hat{\beta}^V_{12} \frac{V X_{12}}{M N}
- \hat{b}^V_{12} X_{12} 
$$

Where:

$$\frac{dX_{12}}{dt}$$

is the rate of increase in coinfected X species ticks

$$\hat{A}_{12} \left(\frac{Y_2 + Y_{12}}{N}\right) X_1$$

represents the increase of coinfected X species ticks from X species ticks solely infected by pathogen 1 becoming coinfected with pathogen 2 after feeding on a pathogen 2 infected host.

$$+ \hat{A}_{21} \left(\frac{Y_1 + Y_{12}}{N}\right) X_2$$

represents the increase of coinfected X species ticks from X species ticks solely infected by pathogen 2 becoming coinfected with pathogen 1 after feeding on a pathogen 2 infected host.

$$+ \mu_{12}^{V} \left(\frac{X_2 + X_{12}}{V}\right) X_1$$

represents the number of species X ticks that were solely infected by pathogen 1 but are now coinfected after co-feeding with a pathogen 2 infected species X tick

$$+ \mu_{21}^{V} \left(\frac{X_1 + X_{12}}{V}\right) X_2$$

represents the number of species X ticks that were solely infected by pathogen 2 but are now coinfected after co-feeding with a pathogen 2 infected species X tick

$$+ \mu_{12}^{W} \left(\frac{Z_2 + Z_{12}}{W}\right) X_1$$

represents the number of species X ticks that were solely infected by pathogen 1 but are now coinfected after co-feeding with a pathogen 2 infected species W tick

$$+ \mu_{21}^{W} \left(\frac{Z_1 + Z_{12}}{W}\right) X_2$$

represents the number of species X ticks that were solely infected by pathogen 2 but are now coinfected after co-feeding with a pathogen 1 infected species W tick

$$- \hat{\beta}^V_{12} \frac{V X_{12}}{M N}$$

represents the loss in coinfected species X ticks due to population dynamics

$$- \hat{b}^V_{12} X_{12} $$

represents the loss in coinfected species X ticks due to tick death
